Здесь лежат мои потуги по сбору датафрейма. Ничего специально не структурировал, это не тот ноутбук, на который нужно прямо смотреть, я воспользовался им один раз, забыл и ушёл. Основные идеи тут есть, какие-то мелочи скорее всего потерялись

К сожалению, книги Евгения Николаевича не распознаны нормально, поэтому надо как-то из этого выкручиваться. Выход первый - забить и распарсить пдфку готовыми библиотеками. Выход так себе, он обрабатывает только один язык, а маэстро у нас полиглот, к тому же обрабатывает не очень, ошибки видно даже невооружённым взглядом, плюсы - он делает всё быстро, по итогу это я сразу отмёл.    
Вариант 2 - прогнать через OCR. Для этого надо превратить пдф в кучу картинок и обработать их тессерактом. Тессеракт ставится через всем известное место, нужно установить его в питоне и отдельно в ОС, нужно прописать отдельно к нему путь, ну и сам процесс обработки довольно долгий. По-моему, у меня заняло примерно час. Зато качество - моё почтение, ошибок очень мало, и распознаны оба алфавита. Ну разве что про греческий я забыл, я не знал, что он там вообще есть

Ставим пререквизиты, если надо

In [33]:
!pip install pdf2image
!pip install opencv-python
!pip install pytesseract
!pip install tesseract
!pip install tesseract-ocr

In [57]:
# прописываем путь к СВОЕМУ тессеракту на СВОЁМ компе
pytesseract.pytesseract.tesseract_cmd = 'C:/OCR/Tesseract/tesseract.exe' # надо менять, если путь другой

In [43]:
import cv2
import pytesseract
from pdf2image import convert_from_path

# сперва можно просто всё импортировать и разбить пдф на странички
# попплер тоже вредный, может понадобиться его отдельно скачать
pages = convert_from_path('maestro_book1.pdf', 100, poppler_path=r"poppler-0.68.0\bin")

In [142]:
def clean(text, num):
    """Обрабатывает текст первой книги следующим образом:
    заменяет все дефисы, разбивает на слова, убирает первые
    два - это верхний колонтинул, убирает последние 2 или 3,
    разнится от чётности странные, это нижний колонтинул"""
    foot = 2 if num % 2 == 0 else 3
    return " ".join(x for x in text.replace("-\n", "").split("\n")[2:-foot])

In [175]:
page_text = []
from tqdm import tqdm

for i in tqdm(range(8, 410)):
    # сохраняем картинку
    pages[i].save(f'out{i}.jpg', 'JPEG')
    # смотрим на картинку и анализируем
    imgcv = cv2.imread(f'out{i}.jpg')
    imgcv = cv2.cvtColor(imgcv, cv2.COLOR_BGR2RGB)
    text = pytesseract.image_to_string(imgcv,  lang='eng+rus')
    # достаём текст и засовываем в список
    page_text.append(clean(text, i-1))
    #9-409

100%|████████████████████████████████████████████████████████████████████████████████| 402/402 [11:10<00:00,  1.67s/it]


In [176]:
# решил не бить на страницы, непонятно, как их клеить, если не руками
full_text = "".join(x for x in page_text)

# сохраняем результат
with open("maestro_book1.txt", "w", encoding="utf-8") as g:
    g.write(full_text)

Это была первая книга, далее копипаста, всё то же самое для второй, только со своей спецификой. Точно так же чищу дефисы, хедеры, футеры, пишу в файл. Единственная разница - он уже распознан, тессеракт будет оверкиллом, можно делать через пдфридер, заодно будет указан второй способ

In [ ]:
pages = convert_from_path('maestro_book2.pdf', 100, poppler_path=r"poppler-0.68.0\bin")

In [296]:
def clean2(text, num):
    header = 3 if num % 2 == 0 else 7
    text = " ".join(x for x in text.split()[header:] if x != "")
    text = re.sub(r"[\s]{2,}", "", text.replace("\xad", " "))
    return text

In [302]:
from PyPDF2 import PdfReader

reader = PdfReader("maestro_book2.pdf")
text = " ".join(x for x in reader.pages[8].extract_text().split()[2:])
text = re.sub(r"[\s]{2,}", "", text.replace("\xad", " "))  # самая первая страница - красивая
for num in tqdm(range(9, 865)):
    text += clean2(reader.pages[num].extract_text(), num-1)
    
with open("maestro_book2.txt", "w", encoding="utf-8") as g:
    g.write(text)

100%|████████████████████████████████████████████████████████████████████████████████| 856/856 [01:12<00:00, 11.74it/s]


Теперь надо придумать, как вытащить статьи Евгения Николаевича. Опять писать краулер, получается, как это надоело, если честно...

In [307]:
import urllib.request
import requests
import re
import os
from bs4 import BeautifulSoup

Со странички в вк уже удобно доступны все посты

In [448]:
url = "https://vk.com/@evgenyponasenkov"

session = requests.session()
session.trust_env = False

req = session.get(url)
page = req.text
soup = BeautifulSoup(page, 'html.parser')

# все переходы на посты лежат в "а"
a = soup.find_all('a', href=True)

In [438]:
import pandas as pd

# промежуточный датафрейм для вк
df = pd.DataFrame(columns=["url", "title", "date", "text"])

In [450]:
hrefs = ["https://vk.com"+i["href"] for i in a[3:-1]]  # всё, что вытащилось

In [451]:
for href in tqdm(hrefs):
    req = session.get(href)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    text = " ".join(x.text for x in soup.find_all("p"))
    title = soup.find("h1").text
    # поскольку вк писали люди, достать оттуда всю метаинфу несложно

    # мне лень это всё в группы клеить
    date = re.findall(r".{4}-.{2}-.{2}",
                      re.findall(r'"datePublished".*T', page)[0])[0]
    df = pd.concat([df, pd.DataFrame({"url": href, "title": title,
                                      "date": date, "text": text}, index=[0])])


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  6.05it/s]


In [452]:
df

,url,title,date,text
0,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011),2021-02-24,"Начну, простите, с конца. Да, митинги 10-го и ..."
0,https://vk.com/@evgenyponasenkov-istorik-evgen...,Историк Евгений Понасенков: Россия - это ходяч...,2021-02-17,"Историк, режиссер, оперный певец и общественны..."
0,https://vk.com/@evgenyponasenkov-dva-gogolya-i...,"Два Гоголя, или Неизбежность поражения",2021-02-14,"Не секрет, что общество, эпоха, семья - да, чт..."
0,https://vk.com/@evgenyponasenkov-otprazdnuem-k...,Отпразднуем капитуляцию кремлевского гарнизона...,2021-01-27,"Начну не с того, что правительство России не в..."
0,https://vk.com/@evgenyponasenkov-gosudarstvenn...,Государственный переворот и «НТВшники» (10. 04...,2021-01-21,"Как это ни печально, но скоро основатель движе..."
0,https://vk.com/@evgenyponasenkov-1812-ubileiny...,"1812: юбилейный зомбоящик, верни дворянам масл...",2020-11-30,Превеликая кампания по зомбированию населения ...
0,https://vk.com/@evgenyponasenkov-1812-god-otve...,1812 год: ответ историка Патриарху Кириллу (10...,2020-11-26,В недавнем азартном отмечании юбилея войны 181...
0,https://vk.com/@evgenyponasenkov-evgenialnyi-s...,ЕвГениальный список архитекторов,2020-09-16,"Альбомы по всей архитектуре Древнего Египта, Г..."
0,https://vk.com/@evgenyponasenkov-intelligentsk...,Интеллигентский мазохизм — страшнее терроризма...,2020-08-17,"Господа-товарищи, всему же есть пределы! Вчера..."
0,https://vk.com/@evgenyponasenkov-chudovischnye...,Чудовищные уроки 1917 года: убийство как истор...,2020-08-09,"Надо признать, что в начале прошлого века коре..."


In [588]:
df = pd.DataFrame(columns=["url", "title", "date", "text"])

In [589]:
for i in range(1, 2):

    url = f"https://www.mk.ru/blogs/blog-evgeniya-ponasenkova.html?page={i}"

    session = requests.session()
    session.trust_env = False

    req = session.get(url)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    a = soup.find_all('a', href=True)
    refs = []
    for i in a:
        href = i["href"]
        if "https://www.mk.ru/blogs/posts/" in href:
            refs.append(href)

    for href in refs[:-5]:  # 5 последних на всех страницах
        sub_req = session.get(href)
        sub_page = sub_req.text
        sub_soup = BeautifulSoup(sub_page, 'html.parser')
        text = " ".join(x.text for x in sub_soup.find_all("p"))
        text = text.replace("\n", " ").split()
        text = text[:-275] # это реклама
        title = sub_soup.find("h1").text + ". Блог Евгения Понасенкова на mk.ru"
        date = " ".join(text[i] for i in range(3))
        text = " ".join(x for x in text[8:])  # до этого не важная информация
        df = pd.concat([df, pd.DataFrame({"url": href, "title": title, "date": date, "text": text}, index=[0])])

Где-то тут я опомнился, что я не записал тексты книг в датафреймы.

In [ ]:
with open("maestro_book2.txt", "r", encoding="utf-8") as f:
    text = f.read()

df = pd.concat([df, pd.DataFrame({"url": "https://www.moscowbooks.ru/bookinist/book/308360/",
                                  "title": 'Первая научная история войны 1812 года',
                                  "date": 2018,
                                  "text": text}, index=[0])])

with open("maestro_book1.txt", "r", encoding="utf-8") as f:
    text = f.read()

df = pd.concat([df, pd.DataFrame({"url": "https://www.ozon.ru/product/pervaya-nauchnaya-istoriya-voyny-1812-goda-trete-izdanie-ponasenkov-evgeniy-nikolaevich-250460112/?sh=cAScFiM5SQ",
                                  "title": 'Правда о войне 1812 года',
                                  "date": 2004,
                                  "text": text}, index=[0])])



In [592]:
from IPython.display import clear_output
from tqdm import tqdm
tqdm.pandas()

In [593]:
def preprocess(text):
    # идея была в том, чтобы присвоить токену ключ,
    # поскольку я кучу раз применяю функцию, нужно хранить счётчик вне
    global i
    # параллельно заполняется база морфохарактеристик,
    # которые, как выяснилось, не нужны, но я засуну куда-нибудь
    global df2

    # токенизирую, вырезаю токены, делаю морфоразбор
    lowered = tokenizer.tokenize(text.lower())
    forms = predictor.predict(lowered)
    clear_output()  # почему нет verbose у предиктора вне моего понимания
    tokens = {}  # словарь токенов предложения, для максимальной скорости
    lemmas = {}  # леммы аналогично

    for form in forms:
        word = form.word
        lemma = form.normal_form

        # делаю ссылки, если слово есть, то она одна, если
        # несколько, будет ссылаться на несколько слов
        # может быть, избыточно
        if word in tokens:
            tokens[word] += [i]
        else:
            tokens.update({word: [i]})

        if lemma in lemmas:
            lemmas[lemma] += [i]
        else:
            lemmas.update({lemma: [i]})

        df2 = pd.concat([df2, pd.DataFrame({"token": word,
                                            "lemma": lemma,
                                            "pos": form.pos,
                                            "tag": form.tag
                                           },
                                            index=[i])])
        i += 1

    # возвращаю словарь, чтобы удобно побить на датафрейм
    return {"tokens": tokens, "lemmas": lemmas}

In [594]:
df = pd.read_csv("sent_combined.csv", index_col=0)
df.head()

,url,title,date,sentence
0,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Начну, простите, с конца."
1,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Да, митинги 10-го и 24-го декабря – это замеча..."
2,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,У России был один приближенный к серьезному ша...
3,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Немного снижая пафос, можно было бы вспомнить ..."
4,https://vk.com/@evgenyponasenkov-kutuzovskii-p...,Кутузовский план Кремля (27. 12. 2011). Запись...,2021-02-24,"Собственно на этом я бы и завершил статью, одн..."


In [598]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'[^\d\W]+')
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language="ru")

INFO:pymorphy2.opencorpora_dict.wrapper:Loading dictionaries from C:\Users\Maloy\AppData\Local\Programs\Python\Python310\lib\site-packages\pymorphy2_dicts_ru\data
INFO:pymorphy2.opencorpora_dict.wrapper:format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345
INFO:pymorphy2.opencorpora_dict.wrapper:Loading dictionaries from C:\Users\Maloy\AppData\Local\Programs\Python\Python310\lib\site-packages\pymorphy2_dicts_ru\data
INFO:pymorphy2.opencorpora_dict.wrapper:format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [604]:
i = 0
df2 = pd.DataFrame({"token":[], "lemma":[] , "pos":[], "tag":[]})

#t = pd.DataFrame(df.sentence.iloc[:10].progress_apply(preprocess).values)
t = pd.DataFrame(df.sentence.progress_apply(preprocess).values)
# обрабатываю датафрейм, запихиваю результат отдельно, чтобы отладить
t["lemmas"] = t[0].apply(lambda x: x["lemmas"])
t["tokens"] = t[0].apply(lambda x: x["tokens"])
t = t.drop(0, axis=1)

 21%|███████████████▌                                                           | 5473/26343 [25:18<1:36:31,  3.60it/s]


MemoryError: Unable to allocate 3.29 MiB for an array with shape (4, 107659) and data type object

In [603]:
df2

,token,lemma,pos,tag
0,начну,начать,VERB,Mood=Ind|Number=Sing|Person=1|Tense=Notpast|Ve...
1,простите,простить,VERB,Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voi...
2,с,с,ADP,_
3,конца,конец,NOUN,Case=Gen|Gender=Masc|Number=Sing
4,да,да,CONJ,_
...,...,...,...,...
10441,во,в,ADP,_
10442,всех,весь,DET,Case=Loc|Number=Plur
10443,сословиях,сословие,NOUN,Case=Loc|Gender=Neut|Number=Plur
10444,и,и,CONJ,_


In [ ]:
def word_search(word, df=df, df2=df2):
    v = np.vectorize(lambda x: word in x)
    indices = np.where(v(df["tokens"]) == True)[0]
    
    for i in indices:
        print(i)
        print(df.iloc[i]["sentence"])
        morph = df2.iloc[i]
        print(f"""\tnormal form - {morph.lemma}
        pos - {morph.pos}
        tag - {morph.tag}""")
        print("")